In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = "D:/Valid"

CATEGORIES = ["Dog", "Cat"]

for category in CATEGORIES:  # do dogs and cats
    path = os.path.join(DATADIR,category)  # create path to dogs and cats
    for img in os.listdir(path):  # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        IMG_SIZE = 100
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        plt.imshow(new_array, cmap='gray')
        plt.show()
       

        break  
    break  

<Figure size 640x480 with 1 Axes>

In [2]:
training_data = []

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
          

create_training_data()

print(len(training_data))

100%|███████████████████████████████████████████████████████████████████████████████| 501/501 [00:00<00:00, 861.06it/s]


1001


In [3]:
import random

random.shuffle(training_data)

for sample in training_data[:10]:
    print(sample[1])

0
0
0
1
1
0
0
0
1
1


In [4]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

[[[[146]
   [155]
   [152]
   ...
   [183]
   [185]
   [187]]

  [[168]
   [172]
   [180]
   ...
   [176]
   [180]
   [175]]

  [[161]
   [156]
   [168]
   ...
   [169]
   [175]
   [174]]

  ...

  [[130]
   [128]
   [153]
   ...
   [124]
   [113]
   [118]]

  [[124]
   [125]
   [137]
   ...
   [127]
   [120]
   [115]]

  [[111]
   [122]
   [131]
   ...
   [133]
   [122]
   [113]]]]


In [5]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)
#print(X)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])

model.save('64x3-CNN.model')

3-conv-64-nodes-0-dense-1549442687
Train on 700 samples, validate on 301 samples
Epoch 1/10
700/700 [==============================] - ETA: 16s - loss: 0.6972 - acc: 0.43 - ETA: 14s - loss: 0.6919 - acc: 0.50 - ETA: 12s - loss: 0.7052 - acc: 0.46 - ETA: 11s - loss: 0.7008 - acc: 0.48 - ETA: 10s - loss: 0.7000 - acc: 0.47 - ETA: 10s - loss: 0.6987 - acc: 0.47 - ETA: 9s - loss: 0.6975 - acc: 0.5089 - ETA: 8s - loss: 0.6973 - acc: 0.492 - ETA: 8s - loss: 0.6966 - acc: 0.506 - ETA: 7s - loss: 0.6970 - acc: 0.490 - ETA: 6s - loss: 0.6965 - acc: 0.491 - ETA: 6s - loss: 0.6961 - acc: 0.497 - ETA: 5s - loss: 0.6960 - acc: 0.488 - ETA: 4s - loss: 0.6958 - acc: 0.488 - ETA: 4s - loss: 0.6956 - acc: 0.483 - ETA: 3s - loss: 0.6953 - acc: 0.490 - ETA: 2s - loss: 0.6951 - acc: 0.490 - ETA: 2s - loss: 0.6951 - acc: 0.489 - ETA: 1s - loss: 0.6953 - acc: 0.488 - ETA: 1s - loss: 0.6952 - acc: 0.489 - ETA: 0s - loss: 0.6955 - acc: 0.483 - 15s 21ms/step - loss: 0.6955 - acc: 0.4800 - val_loss: 0.6930 - va

In [7]:
#TESTING THE MODEL

import cv2
import tensorflow as tf

CATEGORIES = ["Dog", "Cat"]


def prepare(filepath):
    IMG_SIZE = 100  
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("64x3-CNN.model")

prediction = model.predict([prepare('cat.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

[[1.]]
Cat
